In [1]:
# Import packages
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
import netCDF4 as nc

In [2]:
# Import this file to derive the polygon geometry for Telangana
bound_Telangana = gpd.read_file('C:/Users/mieke/Documents/Repositories GitHub/dicra/analytics/geospatial_internship/datasets/telangana_shapefile.geojson')
bound_Telangana.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"MULTIPOLYGON (((79.21147 16.36281, 79.22324 16..."


In [3]:
# Create a Polygon geometry of Telangana
bound_Telangana = bound_Telangana.explode()
bound_Telangana.head()

C:\Users\mieke\AppData\Local\Temp\ipykernel_21968\3231266139.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  bound_Telangana = bound_Telangana.explode()


,,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"POLYGON ((79.21147 16.36281, 79.22324 16.35036..."


In [4]:
# Import the netCDF data for Leaf Area Index
nc_file = gdal.Open('NETCDF:C:/Users/mieke/Downloads/c_gls_LAI300-RT0_202205310000_GLOBE_OLCI_V1.1.2.nc:LAI')
# Convert the netCDF to Geotiff file
tif_file = gdal.Translate('C:/Users/mieke/Downloads/nc_to_tiffile.tif', nc_file)

In [5]:
# Print some metadata related to the Leaf Area Index
nc_metadata = nc.Dataset('C:/Users/mieke/Downloads/c_gls_LAI300-RT0_202205310000_GLOBE_OLCI_V1.1.2.nc')
nc_metadata['LAI']

<class 'netCDF4._netCDF4.Variable'>
uint8 LAI(time, lat, lon)
    _FillValue: 255
    standard_name: leaf_area_index
    long_name: Leaf Area Index 333m
    grid_mapping: crs
    scale_factor: 0.0333333333
    add_offset: 0.0
    units: m^2/m^2
    valid_range: [  0 210]
    missing_value: 255
unlimited dimensions: time
current shape = (1, 47040, 120960)
filling on

In [6]:
# In the next two code blocks, we crop the Geotiff file to Leaf Area Index in Telangana
LAI_data = "C:/Users/mieke/Downloads/nc_to_tiffile.tif"
with rasterio.open(LAI_data) as src:
    out_image, out_transform = rasterio.mask.mask(src, bound_Telangana.geometry, crop=True)
    out_meta = src.meta

In [12]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save the Geotiff file to Leaf Area Index in Telangana
with rasterio.open("C:/Users/mieke/Downloads/Telangana_LAI.tif", "w", **out_meta) as dest:
    dest.write(out_image)

CPLE_AppDefinedError: C:/Users/mieke/Downloads/Telangana_LAI.tif: MissingRequired:TIFF directory is missing required "StripOffsets" field